## Answers to Assignment 2 of 230A

##### Authors(alphabetic order)
1) Ishan Pathak ishan_pathak@berkeley.edu
2) Leonid Maksymenko leo.maksymenko@berkeley.edu
3) Nymph Qiu nymph_qiu@berkeley.edu
4) Xuanzhong Ren rxz19@berkeley.edu

### Prerequisite

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Q1

In [158]:
cov=pd.read_csv('CovarianceMatrix.csv',header=None)
expected_excess_return=pd.read_csv('ExpectedExcessReturns.csv',header=None)
market_excess_return=pd.read_csv('MarketExcessReturns.csv',header=None)
monthly_excess_returns=pd.read_csv('MonthlyExcessReturns.csv',header=None)
expected_excess_return=np.array(expected_excess_return).reshape(-1,1)
cov=np.array(cov)

#### (a)

##### (i)

$$ min_{\alpha}\ \frac{1}{2}\alpha^{'}\Sigma\alpha$$
$$ st.\ \mu_{0}=r_{f}+\alpha^{'}(\mu-r_{f}\mathbf{1})$$
We get,
$$
\alpha = \frac{\mu_0 - r_f}{A r_f^2 - 2 r_f B + C r_f^2}\Sigma^{-1} (\mu - r_f \mathbf{1})
$$
$$
\frac{1}{2}\alpha^{'}\Sigma\alpha=\frac{\mu_0 - r_f}{A r_f^2 - 2 r_f B + C r_f^2}\sqrt{(\mu-r_{f})^T\Sigma^{-1}(\mu-r_{f})}
$$
$$Sharpe=\sqrt{(\mu-r_{f})^T\Sigma^{-1}(\mu-r_{f})}$$
We can see the sharpe ratio does not depend on $\mu_0$, and given $\mu_0$, the numerator in sharpe is fixed. When we try to minimize $\sigma$, equivalently we are trying to maximize the sharpe.

##### (ii)

In the slides, the formula is:
$$ w^*=\frac{\Sigma^{-1}(\mu-r_f \mathbf{1})}{B-Ar_f}$$
where $$ A=\mathbf{1}^{'}\Sigma^{-1}\mathbf{1},B=\mathbf{1}^{'}\Sigma^{-1}\mu$$
we take $$\mu_e=\mu-r_f\mathbf{1}$$
we can see $$w^*=\frac{\Sigma^{-1}\mu_e}{\mathbf{1}^{'}\Sigma^{-1}\mu_e}$$

#### (b)

In [159]:
estimate_miu_rf=np.array(monthly_excess_returns.mean()).reshape(-1,1)
estimate_cov=np.cov(monthly_excess_returns.T)
market_excess_return=np.array(market_excess_return).reshape(-1,1)
monthly_excess_returns=np.array(monthly_excess_returns)
stock_num=len(cov)
rf=0.002

##### (i)

In [131]:
def sharpe(weight,miu,cov):
    return ((miu.T@weight)/np.sqrt(weight.T@cov@weight))

In [160]:
def calculate_sharpe_ratio(expected_excess_return,cov):
    inverse_cov=np.linalg.inv(cov)
    return np.sqrt(expected_excess_return.T@inverse_cov@expected_excess_return)[0,0]
calculate_sharpe_ratio(expected_excess_return,cov)


0.15864241339269422

#### (ii)

In [133]:

def avg_portfolio(estimate_miu_rf, estimate_cov):
    ones = np.ones((stock_num,1))
    w=ones/stock_num
    estimate_portfolio_S1N=sharpe(w,expected_excess_return,cov)
    print('Sharpe ratio of 1/N portfolio: ', estimate_portfolio_S1N[0,0])
    return estimate_portfolio_S1N[0,0]
avg_portfolio(estimate_miu_rf, estimate_cov)

Sharpe ratio of 1/N portfolio:  0.15768721724545703


0.15768721724545703

#### (iii)

In the slides, the optimal sharpe ratio formula is:
$$ w^*=\frac{\Sigma^{-1}(\mu-r_f \mathbf{1})}{B-Ar_f}$$
where $$ A=\mathbf{1}^{'}\Sigma^{-1}\mathbf{1},B=\mathbf{1}^{'}\Sigma^{-1}\mu$$

In [149]:
def optimal(estimate_cov,estimate_miu_rf):
    ones=np.ones((stock_num,1))
    inverse_cov=np.linalg.inv(estimate_cov)
    w=(inverse_cov@estimate_miu_rf)/ones.T@inverse_cov@estimate_miu_rf
    ans=sharpe(w,expected_excess_return,cov)
    return ans[0,0]
optimal(estimate_cov,estimate_miu_rf)

0.0630945101419683

#### (iv)

$$ min_w \ w^{T}\Sigma^{-1}w$$
$$st. \ w^{T}\mathbf{1}=1$$
we get $$w=\frac{\Sigma^{-1}\mathbf{1}}{\mathbf{1^{T}}\Sigma{-1}\mathbf{1}}$$
$$ vol_{min}=\frac{1}{\mathbf{1}\Sigma^{-1}\mathbf{1}}

In [161]:
def min_variance(estimate_cov):
    ones=np.ones((len(estimate_cov),1))
    inverse_cov=np.linalg.inv(estimate_cov)
    vol_min = 1/(ones.T@inverse_cov@ones)
    weight=vol_min*inverse_cov@ones
    ans=sharpe(weight,expected_excess_return,cov)
    print('Sharpe ratio for minimum variance portfolio: ', ans[0,0])
    return ans[0,0]
min_variance(estimate_cov)


Sharpe ratio for minimum variance portfolio:  0.08100024260380302


0.08100024260380302

#### (v)

In [162]:
def jsj(estimate_miu_rf,estimate_cov):
    ones=np.ones((stock_num,1))
    t=360
    original=estimate_miu_rf+rf
    inverse_cov=np.linalg.inv(estimate_cov)
    mu_0=ones.T@inverse_cov@original/(ones.T@inverse_cov@ones)[0,0]
    v=(stock_num+2)/(stock_num+2+t*(original-mu_0*ones).T@inverse_cov@(original-mu_0*ones))[0,0]
    mu_jsj=(1-v)*estimate_miu_rf+v*mu_0*ones-rf

    sharpe_ratio_jsj=optimal(estimate_cov,mu_jsj)
    print('Sharpe ratio of optimal portfolio with adjustment: ', sharpe_ratio_jsj)
    return sharpe_ratio_jsj
jsj(estimate_miu_rf,estimate_cov)

Sharpe ratio of optimal portfolio with adjustment:  0.06295575188031849


0.06295575188031849

#### (vi)

In [164]:
def capm(market_excess_return,monthly_excess_returns,estimate_cov):
    def calc_capm_beta(market_excess_return, stock_excess_return):
        beta=[]
        for i in range(stock_excess_return.shape[1]):
            beta.append(np.cov(stock_excess_return[:,i], market_excess_return[:,0])[0,1] / np.var(market_excess_return))
        return beta
    ones=np.ones((stock_num,1))
    beta=calc_capm_beta(market_excess_return, monthly_excess_returns)
    capm_expected_excess_return =(np.array(beta) * market_excess_return.mean()).reshape(stock_num,1)
    capm_sharpe_ratio=optimal(estimate_cov,capm_expected_excess_return)
    print('Sharpe ratio of CAPM optimal portfolio: ', capm_sharpe_ratio)
    return capm_sharpe_ratio
capm(market_excess_return,monthly_excess_returns,estimate_cov)

Sharpe ratio of CAPM optimal portfolio:  0.15864241339269436


0.15864241339269436

#### (vii)

The optimization problem can be written as:
$$ max_{\alpha} \frac{(\alpha^{T}\mu_e)^2}{\alpha^{T}\Sigma\alpha}$$
$$st.\ \alpha^{T}\mathbf{1}=1$$
$$ \alpha\geqslant\mathbf{0}$$

In [152]:
from scipy.optimize import minimize
def c_mv(estimate_miu_rf,estimate_cov):
    def negative_sharpe_ratio_squared(x, mu_e, Sigma):
        alpha = x.reshape(-1,1)
        portfolio_return = (alpha.T @ mu_e)
        portfolio_volatility = np.sqrt(alpha.T @Sigma @ alpha)
        sharpe_ratio_squared = portfolio_return / portfolio_volatility
        return -sharpe_ratio_squared[0]

    initial_guess = 1 / stock_num * np.ones(stock_num)
    bounds = [(0, 1)]*stock_num 
    constraints = (
        {'type': 'eq', 'fun': lambda x: 1-np.sum(x)},  
    )
    result = minimize(negative_sharpe_ratio_squared, initial_guess, args=(estimate_miu_rf, estimate_cov),
                    method='SLSQP', bounds=bounds, constraints=constraints)
    alpha_optimal = result.x
    print(np.sum(alpha_optimal))
    print('S_c_mv: ', -negative_sharpe_ratio_squared(alpha_optimal, estimate_miu_rf, estimate_cov)[0])
    return -negative_sharpe_ratio_squared(alpha_optimal, estimate_miu_rf, estimate_cov)[0]
c_mv(estimate_miu_rf,estimate_cov)

1.0000000000000007
S_c_mv:  0.24049039890645454


0.24049039890645454

#### 3

In [112]:

N = 50  
T = 360 
q = 0.05 
rf = 0.002 
rm = 0.0008 
def simulate_returns():
    beta_n = np.random.uniform(0.5, 1.5, N)
    alpha_n = np.random.uniform(0.1, 0.3, N)
    idiosyncratic_shock = np.random.normal(0, 1, (T, 1))
    returns = rf + beta_n * rm + alpha_n * idiosyncratic_shock
    cov_matrix = np.diag(alpha_n**2) + q**2 * np.outer(beta_n, beta_n)
    expected_returns = rf + rm * beta_n
    
    return returns, cov_matrix, expected_returns, beta_n

returns, cov_matrix, expected_returns, beta_n = simulate_returns()


In [153]:

avg_1n=[]
optimal_portfolio=[]
min_variance_portfolio=[]
jsj_portfolio=[]
capm_portfolio=[]
cmv=[]
simulation_num=10000
market_excess_return_simulation=np.array([0.0008]*360).reshape(-1,1)
for _ in range(simulation_num):
    returns, cov_matrix_simulation, expected_returns_simulation, beta_n = simulate_returns()
    expected_excess_return_simulation=(returns.mean(axis=0)-rf).reshape(-1,1)
    expected_excess_return=expected_excess_return_simulation
    cov= cov_matrix_simulation
    stock_excess_return_simulation=returns-rf
    avg_1n.append(avg_portfolio(expected_excess_return_simulation,cov_matrix_simulation))
    optimal_portfolio.append(optimal(cov_matrix_simulation,expected_excess_return_simulation))
    min_variance_portfolio.append(min_variance(cov_matrix_simulation))
    jsj_portfolio.append(jsj(expected_excess_return_simulation,cov_matrix_simulation))
    capm_portfolio.append(capm(market_excess_return_simulation,stock_excess_return_simulation, cov_matrix_simulation))
    # cmv.append(c_mv(expected_excess_return_simulation,cov_matrix_simulation))


Sharpe ratio of 1/N portfolio:  -0.0765922556433695
Sharpe ratio for minimum variance portfolio:  -0.07860095805469543
Sharpe ratio of optimal portfolio with adjustment:  -0.07916658646073903
Sharpe ratio of CAPM optimal portfolio:  -0.007553348999321503
Sharpe ratio of 1/N portfolio:  0.20166198983239575
Sharpe ratio for minimum variance portfolio:  0.19738509585731465
Sharpe ratio of optimal portfolio with adjustment:  0.20253985936319266
Sharpe ratio of CAPM optimal portfolio:  -0.010447718504423067
Sharpe ratio of 1/N portfolio:  -0.01780594115307711
Sharpe ratio for minimum variance portfolio:  -0.018192365183692855
Sharpe ratio of optimal portfolio with adjustment:  -0.018278683736893012
Sharpe ratio of CAPM optimal portfolio:  0.0032712510012083947
Sharpe ratio of 1/N portfolio:  0.15539876109959028
Sharpe ratio for minimum variance portfolio:  0.14654064019721758
Sharpe ratio of optimal portfolio with adjustment:  0.14899373542736682
Sharpe ratio of CAPM optimal portfolio:  0.0

In [165]:
print('Average Sharpe ratio of 1/N portfolio: ', np.mean(avg_1n))
print('Average Sharpe ratio of optimal portfolio: ', np.mean(optimal_portfolio))
print('Average Sharpe ratio of minimum variance portfolio: ', np.mean(min_variance_portfolio))
print('Average Sharpe ratio of JSJ portfolio: ', np.mean(jsj_portfolio))
print('Average Sharpe ratio of CAPM portfolio: ', np.mean(capm_portfolio))


Average Sharpe ratio of 1/N portfolio:  0.015115822146642713
Average Sharpe ratio of optimal portfolio:  0.011691840867747565
Average Sharpe ratio of minimum variance portfolio:  0.013253703190146331
Average Sharpe ratio of JSJ portfolio:  0.01314565471691264
Average Sharpe ratio of CAPM portfolio:  0.0002629410654678795


JSJ estimtates are similar to optimal portfolio. Capm behaves the worst. In addition, the optimal portfolio performance is even worse than MVP and 1/N, which means that there exists much discrepancy between the estimates and the true value.

### Q2

(a)

$$E_1(u(W_2))=-e^{-\gamma W_1-\gamma(\alpha_1^1\mu_1^1+\alpha_1^2\mu_1^2)+\frac{1}{2}\gamma^2((\alpha_1^1\sigma^2)^2+(\alpha_2^2\sigma^2)^2)}$$

$$argmax_{\alpha_1^1,\alpha_1^2}-\gamma(\alpha_1^1\mu_1^1+\alpha_1^2\mu_1^2)+\frac{1}{2}\gamma^2((\alpha_1^1\sigma^2)^2+(\alpha_2^2\sigma^2)^2)$$

$$\alpha_1^1=\frac{\mu_1^1}{\gamma(\sigma^2)^2}, \alpha_1^2=\frac{\mu_1^2}{\gamma(\sigma^2)^2}=0$$

So maximum expected utility @t=1 is<br>
$$\max E_1(u(W_2))=-e^{-\gamma W_1-\frac{q-k\tilde{\mu}_0^2}{2(\sigma^2)^2}}$$
$$E_0(u(W_2))=E_0(-e^{-\gamma W_0-\gamma(\alpha_0^1\tilde{\mu}_0^1+\alpha_0^2\tilde{\mu}_0^2)-\frac{q-k\tilde{\mu}_0^2}{2(\sigma^2)^2}})=E_0(-e^{-\gamma W_0-\gamma(\alpha_0^1\tilde{\mu}_0^1+\alpha_0^2\tilde{\mu}_0^2)-\frac{q-k\tilde{\mu}_0^2}{2(\sigma^2)^2}})=-e^{-\gamma W_0-\frac{q}{2(\sigma^2)^2}}E_0(e^{-\gamma(\alpha_0^1\tilde{\mu}_0^1+\alpha_0^2\tilde{\mu}_0^2)+\frac{k\tilde{\mu}_0^2}{2(\sigma^2)^2}})$$
$$\alpha_0^1=\frac{\mu_0^1}{\gamma(\sigma^1)^2}=\frac{q}{\gamma(\sigma^1)^2}, \alpha_0^2=\frac{k}{2\gamma(\sigma^2)^2}$$

(b)

k is like a scalar for impact on expected return of asset 1 from realized return of asset 2 @t=0. This correlation would impact the investment decisions made per period for asset 1 & 2.<br>
When k=0, there is zero investment in asset 2 @t=0. And asset 1 return would not be impacted by asset 2 so per period, asset 1 and 2 contribute to maximum expected utility individually.<br>
When k>0, there would be positive investment in asset 2. When realized return of $\tilde{\mu}_0^2$ goes up, expected return for $\tilde{\mu}_1^1$ goes down, which would reduce asset 1 investment @t=1.

(c)

Dollar investment $\alpha_1^2$ is not impacted by $\tilde{\mu}_0$. So when $k\neq 0$, $\alpha_1^2$ is zero anyway. <br>
Dollar investment $\alpha_1^1$ would be impacted. The expected return for asset 1 @t=1 is $q-k\tilde{\mu}_0^2$ given the realized return. If k>0, when $\tilde{\mu}_0^2$ goes up, it reduces expected return for asset 1 @t=1 which in response, $\alpha_1^1$ is reduced as well. Otherwise, when $\tilde{\mu}_0^2$ goes up, expected return increases then $\alpha_1^1$ is augmented as well.<br>
Risky-asset weight $\omega_1^1, \omega_1^2$ would be impacted due to $\alpha_1^1$. If k > 0, when $\tilde{\mu}_0^2$ goes up, $\alpha_1^1$ is reduced so $\omega_1^1$ goes down while $\omega_1^2$ increases. If k < 0, when $\tilde{\mu}_0^2$ goes down, $\alpha_1^1$ is increased, $\omega_1^1$ goes up while $\omega_1^2$ decreases.